In [1]:
# imports
from subprocess import check_output
import pandas as pd
import plotly.express as px
import plotly
import re
import nbformat
import time
import json

In [5]:
# measure probability calculation efficiency
results_df = pd.DataFrame(columns = ['Processed Cells', 'Retries', 'Time', 'Actions'])
agents = ["Agent 6", "Agent 7", "Agent 8"]

for x in range(1):
  command = 'python main.py -d 50 -p 0.3'
  out = json.loads(str(check_output(command, cwd='/Users/advithchegu/Desktop/Rutgers/Fall 2021/AI/Assignment 3/src', shell=True)).lstrip("b'").rstrip("'").replace("\\n", ""))

  for i,a in enumerate(agents):
    results_df = results_df.append([{'Agent': i+6, 'Processed Cells': out[a]["processed"], 'Retries': out[a]["retries"], 'Time': out[a]["time"], 'Actions': out[a]["actions"]}], ignore_index=True)

results_df.head()

,Processed Cells,Retries,Time,Actions,Agent
0,4154,508,11.873556,1419,6.0
1,175519,4912,35.517588,27838,7.0
2,155169,4935,101.502806,26967,8.0
